In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# model_name = "kaggle_trashbox_10epoch_flip_only"
# model = tf.keras.models.load_model('./trained_model/'+ model_name +".h5")
# # model.summary()

## Create tf.data.dataset object

In [2]:
test_trashbox_val_path = "E:/data/Trashbox/trashbox_val/"
test_trashbox_test_path = "E:/data/Trashbox/trashbox_test/"
test_recycling_path = "E:/data/Recycling/Recycling_test/"
test_recycling_9k_path = "E:/data/Recycling/Recycling_dataset_224_3/"
test_img_real_3_path = "E:/data/Img_real_3/"
test_trashnet_path = "E:/data/TrashNet/"
test_vending_5class_path = "E:/data/UEH_vending/UEH_vending_5class/UEH_vending_5class_test/"

# test_vending_5class = tf.keras.utils.image_dataset_from_directory(
#   test_vending_5class_path,  
#   image_size=(224, 224),
#   label_mode = 'categorical', 
#   shuffle=False,
#   batch_size=32)

# normalization_layer = layers.Rescaling(1./255)
batch_size = 64
#1
test_trashbox_val = tf.keras.utils.image_dataset_from_directory(
  test_trashbox_val_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=batch_size)

class_names = test_trashbox_val.class_names
print(class_names)

# test_trashbox_val = test_trashbox_val.map(lambda x, y: (normalization_layer(x), y))

#2
test_trashbox_test = tf.keras.utils.image_dataset_from_directory(
  test_trashbox_test_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=batch_size)

# test_trashbox_test = test_trashbox_test.map(lambda x, y: (normalization_layer(x), y))

#3
test_recycling = tf.keras.utils.image_dataset_from_directory(
  test_recycling_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=batch_size)

# test_recycling = test_recycling.map(lambda x, y: (normalization_layer(x), y))

#4
test_recycling_9k = tf.keras.utils.image_dataset_from_directory(
  test_recycling_9k_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=batch_size)

# test_recycling_9k = test_recycling_9k.map(lambda x, y: (normalization_layer(x), y))

#5
test_img_real_3 = tf.keras.utils.image_dataset_from_directory(
  test_img_real_3_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=batch_size)

# test_img_real_3 = test_img_real_3.map(lambda x, y: (normalization_layer(x), y))

# test_trashnet = tf.keras.utils.image_dataset_from_directory(
#   test_trashnet_path,  
#   image_size=(224, 224),
#   label_mode = 'categorical', 
#   shuffle=False,
#   batch_size=32)

Found 409 files belonging to 3 classes.
['Alu', 'Glass', 'PET']
Found 412 files belonging to 3 classes.
Found 1840 files belonging to 3 classes.
Found 9200 files belonging to 3 classes.
Found 490 files belonging to 3 classes.


## Evaluate test data

In [3]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def Create_cm(predictions, labels_list, title, test_ds, class_names, cm = False, normalize = None, save = False):

    if cm:
        arr = confusion_matrix(labels_list, predictions, normalize = normalize)   

        df_cm = pd.DataFrame(arr, index = [i for i in class_names],
                          columns = [i for i in class_names])
        plt.figure(figsize=(8,6), dpi=100) # dpi for better resolution

        # Scale up the size of all text
        sn.set(font_scale = 1.1)

        # plt.ticklabel_format(useOffset=False,style='plain', axis='y')
        
        
        if normalize == "true":
            ax = sn.heatmap(df_cm, annot=True, fmt=".3f", cmap='Blues')
        else:
            ax = sn.heatmap(df_cm, annot=True, fmt="d", cmap='Blues')    
        ax.set_title(title, fontsize=14, pad=20)
        plt.ylabel('Actual label')
        ax.set_ylabel("Actual label", fontsize=14, labelpad=20)

        plt.xlabel('Predicted label')
        ax.set_xlabel('Predicted label',fontsize=14, labelpad=20)
        print(classification_report(labels_list, predictions, digits=3))
    if save:
        fig = ax.get_figure()
        fig.savefig(model_name + title + "_3class.jpg")
    

def Evaluate(model, test_ds, title, class_names, cm = False, normalize = None, save = False):

    print(title)
    
    # Predictions are from model.predict, true labels are from test_ds
    result = model.predict(test_ds, verbose = False)
    
    # Predictions
    num_images, num_classes = result.shape
    predictions = []
    for i in range(num_images):
        predictions.append(np.argmax(result[i]))

    # True labels    
    labels =  np.array([])
    label = []
    for _, y in test_ds:
        labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

    labels_list = labels.astype('uint8').tolist()
    
    # Metrics
    acc = accuracy_score(labels_list, predictions)
    f1 = f1_score(labels_list, predictions, average="macro")
    print("{:.3f} {:.3f}".format(acc, f1))
    
    Create_cm(predictions, labels_list, title, test_ds, class_names, cm, normalize, save) 
    
    return (acc, f1)

## Evaluate

In [4]:
# test_trashbox_val, test_trashbox_test, test_recycling_9k, test_img_real_3
# def Evaluate(model, test_ds, title, cm = False, normalize = None, save = False):
# import tensorflow as tf

# test_img_real_3_path = "E:/data/UEH_7class/Val/"

# test_img_real_3 = tf.keras.utils.image_dataset_from_directory(
#   test_img_real_3_path,  
#   image_size=(224, 224),
#   label_mode = 'categorical', 
#   shuffle=False,
#   batch_size=32)

model_list = [
    "model_Inc_Contrast",
    "model_Inc_Rot",
    "model_Inc_Crop", 
    "model_Inc_flip_hor",
    "model_Inc_flip_vert"
]

for model_name in model_list: 
    print("Model: ", model_name)
    model = tf.keras.models.load_model('./trained_model/'+ model_name +".h5")
    
    # acc_4, f1_4 = Evaluate(model, test_vending_5class, "Test", cm = True, normalize = None, save = True)
    # acc_1, f1_1 = Evaluate(model, test_img_real_3, "Img Real", class_names, cm = True, normalize = "true")
    acc_1, f1_1 = Evaluate(model, test_trashbox_test, "Trashbox test", class_names,cm = False, normalize = "true")
    acc_2, f1_2 = Evaluate(model, test_trashbox_val, "Trashbox val", class_names,cm = False, normalize = "true")
    acc_3, f1_3 = Evaluate(model, test_recycling_9k, "Recycling 9k",class_names, cm = False, normalize = "true")
    acc_4, f1_4 = Evaluate(model, test_img_real_3, "Test Img Real", class_names,cm = False, normalize = "true")

    print()
    print("{:.3f}".format(acc_2), "{:.3f}".format(f1_2))
    print("{:.3f}".format(acc_1), "{:.3f}".format(f1_1))
    print("{:.3f}".format(acc_3), "{:.3f}".format(f1_3))
    print("{:.3f}".format(acc_4), "{:.3f}".format(f1_4))
    # print(acc_2, f1_2)
    # print(acc_3, f1_3)
    # print(acc_4, f1_4)

    
    avg_acc = (acc_1 + acc_2 + acc_3 + acc_4)/4
    avg_f1 = (f1_1 + f1_2 + f1_3 + f1_4)/4
    avg = (avg_acc + avg_f1)/2

    print("acc: {:.3f}".format(avg_acc))
    print("f1: {:.3f}".format(avg_f1))
    print("avg: {:.3f}".format(avg))

Model:  model_Inc_Contrast
Trashbox test
0.905 0.878
Trashbox val
0.912 0.884
Recycling 9k
0.838 0.808
Test Img Real
0.782 0.754

0.912 0.884
0.905 0.878
0.838 0.808
0.782 0.754
acc: 0.859
f1: 0.831
avg: 0.845
Model:  model_Inc_Rot
Trashbox test
0.898 0.860
Trashbox val
0.900 0.867
Recycling 9k
0.834 0.804
Test Img Real
0.739 0.720

0.900 0.867
0.898 0.860
0.834 0.804
0.739 0.720
acc: 0.843
f1: 0.813
avg: 0.828
Model:  model_Inc_Crop
Trashbox test
0.908 0.877
Trashbox val
0.905 0.874
Recycling 9k
0.828 0.809
Test Img Real
0.731 0.712

0.905 0.874
0.908 0.877
0.828 0.809
0.731 0.712
acc: 0.843
f1: 0.818
avg: 0.830
Model:  model_Inc_flip_hor
Trashbox test
0.905 0.876
Trashbox val
0.910 0.883
Recycling 9k
0.854 0.829
Test Img Real
0.782 0.756

0.910 0.883
0.905 0.876
0.854 0.829
0.782 0.756
acc: 0.863
f1: 0.836
avg: 0.849
Model:  model_Inc_flip_vert
Trashbox test
0.898 0.856
Trashbox val
0.905 0.873
Recycling 9k
0.838 0.812
Test Img Real
0.763 0.741

0.905 0.873
0.898 0.856
0.838 0.812
0.

In [ ]:
model_name = "kaggle_trashbox_30epoch_flip_oneof_05"
model = tf.keras.models.load_model('./trained_model/'+ model_name +".h5")
# model.summary()

# acc_3, f1_3 = Evaluate(model, test_recycling_9k, "Recycling 9k", cm = True, normalize = "true")
acc_4, f1_4 = Evaluate(model, test_trashbox_val, "TrashBox Val", cm = True, normalize = None, save = True)
# acc_5, f1_5 = Evaluate(model, test_recycling, "RECYCLE", cm = True, normalize = None, save = True)

In [ ]:
avg_acc = (acc_1 + acc_2 + acc_3 + acc_4)/4
avg_f1 = (f1_1 + f1_2 + f1_3 + f1_4)/4
avg = (avg_acc + avg_f1)/2

print("acc: {:.3f}".format(avg_acc))
print("f1: {:.3f}".format(avg_f1))
print("avg: {:.3f}".format(avg))

In [ ]:
model_name = "True_augment_flip_hor_nas_3class_20epoch"
model = tf.keras.models.load_model('./trained_model/'+ model_name +".h5")
#1
[_, acc, precision, recall] = model.evaluate(test_trashbox_val, verbose = False)
f1 = 2*precision*recall/(precision+recall)
print("{:.3f} {:.3f}".format(acc, f1))
#2
[_, acc, precision, recall] = model.evaluate(test_trashbox_test, verbose = False)
f1 = 2*precision*recall/(precision+recall)
print("{:.3f} {:.3f}".format(acc, f1))
#3
# [_, acc, precision, recall] = model.evaluate(test_recycling, verbose = False)
# f1 = 2*precision*recall/(precision+recall)
# print("{:.3f} {:.3f}".format(acc, f1))
#4
[_, acc, precision, recall] = model.evaluate(test_recycling_9k, verbose = False)
f1 = 2*precision*recall/(precision+recall)
print("{:.3f} {:.3f}".format(acc, f1))
#5
[_, acc, precision, recall] = model.evaluate(test_img_real_3, verbose = False)
f1 = 2*precision*recall/(precision+recall)
print("{:.3f} {:.3f}".format(acc, f1))

In [ ]:
[_, acc, precision, recall] = model.evaluate(test_trashbox_val, verbose = False)
print([_, acc, precision, recall])

# QUICK TEST

## Test on 3 classes

Model predicts 4 classes
Test data has only 3 classes
test_ds.class_names returns the class of test data

In [ ]:
test_path = "E:/data/kaggle_3/"

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=32)

test_labels = test_ds.class_names
train_labels = ['Alu', 'Alu_crushed', 'Glass', 'PET']

print("test labels: ", test_ds.class_names)
print("train labels: ", train_labels)


result = model.predict(test_ds)


# Predictions, models returns 4 classes
num_images, num_classes = result.shape
predictions = []
for i in range(num_images):    
    predictions.append(train_labels[np.argmax(result[i])])

    
# True labels, Test data only has 3 classes    
labels =  np.array([])
label = []
for _, y in test_ds:
    labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

labels_list = labels.astype('uint8').tolist()    

for i, label in enumerate(labels_list):
    labels_list[i] = test_labels[label]

print("unique in predictions: ", set(predictions))
print("unique in true labels: ", set(labels_list))

## Covert 4 classes pred to 3 classes

In [ ]:
for i, pred in enumerate(predictions):
    if pred == "Alu_crushed":
        predictions[i] = "Alu"
set(predictions) # unique

In [ ]:
print(predictions[250])
print(labels_list[250])
print("unique in predictions: ", set(predictions))
print("unique in true labels: ", set(labels_list))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

y_pred = predictions
y_true = labels_list
arr = confusion_matrix(y_true, y_pred)   
# print(arr)
# arr = confusion_matrix(correct_labels, rounded_labels)


df_cm = pd.DataFrame(arr, index = [i for i in test_labels],
                  columns = [i for i in test_labels])
plt.figure(figsize=(8,6), dpi=100) # dpi for better resolution

# Scale up the size of all text
sn.set(font_scale = 1.1)

# plt.ticklabel_format(useOffset=False,style='plain', axis='y')
ax = sn.heatmap(df_cm, annot=True,fmt='d', cmap='Blues')
ax.set_title("Confusion Matrix for Kaggle drinking waste", fontsize=14, pad=20)
plt.ylabel('Actual label')
ax.set_ylabel("Actual label", fontsize=14, labelpad=20)

plt.xlabel('Predicted label')
ax.set_xlabel('Predicted label',fontsize=14, labelpad=20)

In [ ]:
print(classification_report(y_true, y_pred))

## Find images not accepted by tensorflow and correct their format

In [ ]:
from pathlib import Path
import imghdr
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# img_dir = test_path + "Glass/glass2486.jpg"
# img_dir = r"E:\trashbox\Alu\beverage_cans256.jpg"
# img = mpimg.imread(img_dir)
# print(type(img))
# plt.imshow(img)
# plt.show()
# mpimg.imsave(r"E:\trashbox\Alu\beverage_cans256.jpg", img)


# data_dir = test_path

# test_trashbox_val_path = "E:/data/trashbox_val/"
# test_trashbox_test_path = "E:/data/trashbox_test/"
# test_recycling_path = "E:/data/Recycling_test/"
# train_ds_path = "E:/data/UEH_vending/UEH_vending_5class/UEH_vending_5class_train/"
# val_ds_path = "E:/data/UEH_vending/UEH_vending_5class/UEH_vending_5class_val/"
# test_ds_path = "E:/data/UEH_vending/UEH_vending_5class/UEH_vending_5class_val/"

train_ds_path = "E:/data/UEH_9class/Train/"
val_ds_path = "E:/data/UEH_9class/Val/"

data_dir = val_ds_path
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
            
            # Save that image as jpg    
            img_dir = filepath                         
            img = mpimg.imread(img_dir)
            mpimg.imsave(img_dir, img)                           
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")
            # Save that image as jpg
            img_dir = filepath                     
            img = mpimg.imread(img_dir)
            mpimg.imsave(img_dir, img)                            

In [ ]:
img_dir = test_path + "Glass/glass2486.jpg"
img_dir = r"E:\trashbox\Alu\beverage_cans256.jpg"
img = mpimg.imread(img_dir)
print(type(img))
plt.imshow(img)
plt.show()
mpimg.imsave(r"E:\trashbox\Alu\beverage_cans256.jpg", img)

# Find incorrect predictions

In [ ]:
# Load data to test

test_img_real_3_path = "E:/data/UEH_7class/Train/"

test_img_real_3 = tf.keras.utils.image_dataset_from_directory(
  test_img_real_3_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=32)

# Load the model
model_name = "InceptionV3_7class_60_epoch"
model = tf.keras.models.load_model('./trained_model/'+ model_name +".h5")

In [ ]:
model.summary()

In [ ]:
acc_1, f1_1 = Evaluate(model, test_img_real_3_path, "7 class train", class_names,cm = True, normalize = "true")

In [ ]:
# Predictions are from model.predict, true labels are from test_ds
result = model.predict(test_img_real_3, verbose = True)

# List of Predictions
num_images, num_classes = result.shape
predictions = []
predictions_conf = []
for i in range(num_images):
    predictions.append(np.argmax(result[i]))
    predictions_conf.append(round(max(result[i]), 3))

# List of True labels    
labels =  np.array([])
label = []
for _, y in test_img_real_3:
    labels = np.concatenate([labels, np.argmax(y.numpy(), axis=-1)])

labels_list = labels.astype('uint8').tolist()

In [ ]:
predictions_conf[100]

In [ ]:
# If that image is predicted incorrectly, it will be saved to the desired class output's folder.
# The image's name will be its incorrect prediction

# Load that same dataset but with batch = None

test_img_no_batch = tf.keras.utils.image_dataset_from_directory(
  test_img_real_3_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=None)

class_names = test_img_no_batch.class_names
    
normalization_layer = layers.Rescaling(1./255)

#1
test_trashbox_val = tf.keras.utils.image_dataset_from_directory(
  test_trashbox_val_path,  
  image_size=(224, 224),
  label_mode = 'categorical', 
  shuffle=False,
  batch_size=32)

class_names = test_trashbox_val.class_names

In [ ]:
class_names

In [ ]:
# Create folder for the class_names


import os

path_incorrect = "E:/data/INCORRECT_PRED/UEH_7class_train/"

for i, class_name in enumerate(class_names):
    os.makedirs(path_incorrect + class_names[i])


In [ ]:
# If that image is predicted incorrectly, it will be saved to the desired class output's folder.
# The image's name will be its incorrect prediction

import matplotlib.pyplot as plt

path_incorrect = "E:/data/INCORRECT_PRED/UEH_7class/"

i = 0
correct = 0
incorrect = 0
for img, _ in test_img_no_batch:
    if predictions[i] == labels_list[i]:
        i += 1
        correct += 1
        continue
    elif predictions[i] != labels_list[i]:        
        # The image's name will be its incorrect prediction and confidence, saved to the desired class output's folder.
        img_name = path_incorrect + str(class_names[labels_list[i]]) + "/" + str(class_names[predictions[i]]) + "_" + str(predictions_conf[i]) + '.jpg'
        image = img.numpy().astype("uint8")
        plt.imsave(img_name, image)
        i += 1
        incorrect += 1
        

In [ ]:
correct/i

In [ ]:
# Total images
i

In [ ]:
# Number of incorrect predictions
incorrect

In [ ]:
# Number of correct predictions
correct